Precisamos instalar primeiro o gym. Executar o seguinte em um notebook Jupyter deve funcionar:

In [1]:
!pip install gym

## Q-Learning com Gym
Depois de instalado, podemos carregar o ambiente e renderizar sua aparência:
Abrindo e mostrando um ambiente pronto do gym que simula o problema do Taxi (versão 3):


In [2]:
import gym #importa o gym
env = gym.make("Taxi-v3").env #carrega o ambiente
env.render() #Renderiza uma estrutura do ambiente (útil na visualização do ambiente)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



O quadrado preenchido representa o táxi, que é amarelo sem passageiro e verde com passageiro.
A barra vertical ("|") representa uma parede que o táxi não pode atravessar.
R, G, Y, B são os locais de coleta e destino possíveis. A letra azul representa o local atual de embarque do passageiro e a letra roxa é o destino atual.

In [3]:
print("Total de Ações {}".format(env.action_space))
print("Total de Estados {}".format(env.observation_space))

Total de Ações Discrete(6)
Total de Estados Discrete(500)


### Alterando o estado
Podemos codificar o estado do agente e fornecê-lo ao ambiente para renderizar no Gym. Lembre-se de que temos o táxi na linha 3, coluna 1, nosso passageiro está no local 2 e nosso destino é o local 0. Usando o método de codificação de estado Taxi-v3, podemos fazer o seguinte:

Use a função env.encode(taxi_linha,taxi_coluna,passageiro_saida,passageiro_chegada)

In [6]:
state = env.encode(3, 1, 2, 0) 
print("Número do estado:", state)
env.s = state
env.render()

Número do estado: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Outra forma de definir o estado. Podemos definir o estado do ambiente manualmente env.env.s usando esse número codificado.

369 correspondente a um estado entre 0 e 499. O ambiente possue 500 estados possíveis


In [7]:
env.s = 369
print("Número do estado:", env.s)
env.render()

Número do estado: 369
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



**A Tabela de Recompensas**
Quando o ambiente Taxi é criado, também é criada uma tabela inicial de Recompensas, chamada `P`. Podemos pensar nisso como uma matriz que tem o número de estados como linhas e o número de ações como colunas, ou seja, 
 s t a t e s X a c t i o n s


In [8]:
env.P[329] #estado atual

{0: [(1.0, 429, -1, False)],
 1: [(1.0, 229, -1, False)],
 2: [(1.0, 349, -1, False)],
 3: [(1.0, 329, -1, False)],
 4: [(1.0, 329, -10, False)],
 5: [(1.0, 329, -10, False)]}

Este dicionário tem a estrutura {action: [(probability, nextstate, reward, done)]}.

O 0-5 corresponde às ações (sul, norte, leste, oeste, embarque, desembarque) que o táxi pode executar em nosso estado atual na ilustração.
Neste env, probabilidade é sempre 1.0.
Este next state é o estado em que estaríamos se tomarmos a ação. 
Todas as ações de movimento têm uma recompensa -1 e as ações pickup (pegar)/dropoff(deixar) têm -10 neste estado particular. Se estivermos em um estado em que o táxi tem um passageiro e está em cima do destino certo, veríamos uma recompensa de 20 na ação.
O done é usado para nos dizer quando conseguimos deixar um passageiro no local certo. 

### Resolvendo usando Ações Aleatórias, sem Aprendizagem por Reforço

Vamos ver o que aconteceria se tentarmos usar a força bruta para resolver o problema sem aprendizagem por reforço.

Como temos nossa tabela P de recompensas padrão em cada estado, podemos tentar fazer com que nosso táxi navegue usando apenas isso.

Vamos criar um loop infinito que corre até que um passageiro chegue a um destino, ou em outras palavras, quando a recompensa recebida for 20.

O método env.action_space.sample() seleciona automaticamente uma ação aleatória de um conjunto de todas as ações possíveis.

In [9]:
env.s = 329  # começa no estado do exemplo acima

epochs = 0   # total de ações realizadas 
penalties = 0   # quantidade de punições recebidas por pegar ou largar no lugar errado

frames = [] # usado para fazer uma animação

done = False

while not done:
    action = env.action_space.sample()  # escolhe aleatoriamente uma ação
    state, reward, done, info = env.step(action)  # aplica a ação e pega o resultado

    if reward == -10:  # conta uma punição
        penalties += 1
    
    # Guarda a sequência para poder fazer a animação depois
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Total de ações executadas: {}".format(epochs))
print("Total de penalizações recebidas: {}".format(penalties))



Total de ações executadas: 20457
Total de penalizações recebidas: 6649


### Mostrando a animação dos movimentos realizados

In [11]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 20457
State: 85
Action: 5
Reward: 20


Não é bom. Nosso agente leva milhares de passos de tempo e faz muitos desembarques errados para entregar apenas um passageiro ao destino certo.

Isso ocorre porque não estamos aprendendo com experiências anteriores. Podemos repetir isso repetidamente e nunca será otimizado. O agente não tem memória de qual ação foi melhor para cada estado, que é exatamente o que o Aprendizado por Reforço fará por nós.

### Aprendendo a resolver usando Q-Learning

Essencialmente, o Q-learning permite que o agente use as recompensas do ambiente para aprender, com o tempo, a melhor ação a ser executada em um determinado estado.

Em nosso ambiente de táxi, temos a tabela de recompensas P, com a qual o agente aprenderá. Ele procura receber uma recompensa por realizar uma ação no estado atual e, em seguida, atualiza um valor Q para lembrar se essa ação foi benéfica.

Os valores armazenados na tabela Q são chamados de valores Q e são mapeados para uma (state, action)combinação.

Um valor Q para uma determinada combinação de estado-ação é representativo da "qualidade" de uma ação realizada a partir desse estado. Melhores valores de Q implicam em melhores chances de obter maiores recompensas.

Por exemplo, se o táxi se depara com um estado que inclui um passageiro em sua localização atual, é altamente provável que o valor Q para pickup seja maior quando comparado a outras ações, como dropoff ou north.


In [11]:
import numpy as np

# Inicialização com a tabela de valores Q
# Primeiro, precisamos criar nossa tabela Q, que usaremos para rastrear estados, 
# ações e recompensas. O número de estados e ações no ambiente Táxi determina o tamanho da nossa mesa.
q_table = np.zeros([env.observation_space.n, env.action_space.n])

import random
from IPython.display import clear_output

# Hiperparâmetros
alpha = 0.1   # taxa de aprendizagem
gamma = 0.6   # fator de desconto
epsilon = 0.1  # chance de escolha aleatória  

# Total geral de ações executadas e penalidades recebidas durante a aprendizagem
epochs, penalties = 0,0
# treinamento do agente
for i in range(1, 100001): # Vai rodar 100000 diferentes versões do problema
    state = env.reset()  # Inicialização aleatoria do ambient
    done = False
    
    while not done:
        #épsilon entra em ação quando geramos um valor aleatório entre 0 e 1 e o 
        #comparamos com nosso épsilon (taxa de exploração), se o valor aleatório for menor,
        #executamos uma ação aleatória de nosso espaço de ação
        #senão olhamos nossa tabela Q atual e tomamos a ação que maximiza a função de valor.
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Escolhe ação aleatoriamente
        else:
            action = np.argmax(q_table[state]) # Escolhe ação com base no que já aprendeu

        #Depois de escolher uma ação, continuamos com ela e medimos a recompensa associada. 
        #Isso é feito usando o método embutido env.step (ação) que faz um movimento de um passo de tempo
        #Ele retorna o próximo estado, a recompensa da ação anterior; 
        #done indica se nosso agente atingiu a meta (o que significaria redefinir o ambiente);
        # info é apenas um diagnóstico de desempenho usado para depuração.
        next_state, reward, done, info = env.step(action) # Aplica a ação
        
        old_value = q_table[state, action]  # Valor da ação escolhida no estado atual
        next_max = np.max(q_table[next_state]) # Melhor valor no próximo estado
        
        # Atualize o valor Q usando a fórmula principal do Q-Learning
        # Finalmente, podemos usar as informações coletadas para atualizar a tabela Q usando a equação de Bellman.
        # Lembre-se de que alfa representa a taxa de aprendizado aqui.
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:  # Contabiliza as punições por pegar ou deixar no lugar errado
            penalties += 1

        state = next_state # Muda de estado
        epochs += 1
        
print("Treinamento finalizado.\n")
print("Total de ações executadas: {}".format(epochs))
print("Total de penalizações recebidas: {}".format(penalties))

Total de ações executadas: 1594949
Total de penalizações recebidas: 48550


### Mostrando a tabela Q para o estado 329
Agora que a tabela Q foi estabelecida com mais de 100.000 episódios, vamos ver quais são os valores Q no estado de nossa ilustração:

In [12]:
env.s = 329
env.render()

q_table[329]

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


array([ -2.48850115,  -2.47061344,  -2.48794422,  -2.4815022 ,
       -10.97219233, -11.02119763])

### Resolvendo o problema com o aprendizado adquirido
Vamos avaliar o desempenho do nosso agente. Não precisamos explorar mais as ações, então agora a próxima ação é sempre selecionada usando o melhor valor Q:

In [13]:
state = 329
epochs, penalties = 0, 0
    
done = False
    
while not done:
     action = np.argmax(q_table[state])
     state, reward, done, info = env.step(action)

     if reward == -10:
        penalties += 1

     epochs += 1

print("Total de ações executadas: {}".format(epochs))
print("Total de penalizações recebidas: {}".format(penalties))

Total de ações executadas: 14
Total de penalizações recebidas: 0


Podemos verificar pela avaliação que o desempenho do agente melhorou significativamente e não incorreu em penalidades, o que significa que realizou as ações corretas de embarque / desembarque com 100 passageiros diferentes.
